In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# prevent tensorflow from using GPU. Otherwise, run out of memory
# https://stackoverflow.com/questions/44552585/prevent-tensorflow-from-accessing-the-gpu
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"

import tensorflow_hub as hub
import tensorflow as tf
import nltk
import regex as re
import emoji as em
from sklearn.model_selection import train_test_split


elmo = hub.Module("https://tfhub.dev/google/elmo/2")

W0411 14:32:47.081992 140051505092416 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [2]:
# Set path
path = '/home/tim/Documents/Sentiment/Data/processed'
file1 = '/home/tim/Documents/Sentiment/Data/processed/dev.tsv'
file2 = 'EI-reg-En-joy-train.txt'


# Create dataframe
df1 = pd.read_csv(os.path.join(path,file1),delimiter="\t")
# df2 = pd.read_csv(os.path.join(path,file2),delimiter="\t")



In [3]:
df = df1
# df = pd.concat([df1,df2])

In [4]:
df.head()

,ID,Tweet,Affect Dimension,Intensity Score
0,2017-En-10264,@xandraaa5 @amayaallyn6 shut up hashtags are c...,anger,0.562
1,2017-En-10072,it makes me so fucking irate jesus. nobody is ...,anger,0.750
2,2017-En-11383,Lol Adam the Bull with his fake outrage...,anger,0.417
3,2017-En-11102,@THATSSHAWTYLO passed away early this morning ...,anger,0.354
4,2017-En-11506,@Kristiann1125 lol wow i was gonna say really?...,anger,0.438


In [5]:
df.shape

(1701, 4)

In [6]:
def preprocess(tweet):
    #define hashtag pattern
    hashtag_define = re.compile ('#')
    #define mention pattern 
    mentions_define = re.compile('@[^\s]+')
    #define link pattern
    link_define = re.compile('https?://[^\s]+')
    # Haystack define
    haystack_define = re.compile('(RT)')
    # define long spaces
    extra_spaces_define = re.compile('\s{2,}')
    #remove hashtags
    tweet_refine = hashtag_define.sub('',tweet)
    # remove mentions
    tweet_refine = mentions_define.sub('',tweet_refine)
    # remove links
    tweet_refine = link_define.sub('',tweet_refine)
    # remove haystack
    tweet_refine = haystack_define.sub('',tweet_refine)
    #replace long spaces with one space
    tweet_refine = extra_spaces_define.sub(' ',tweet_refine)
    # convert emoticons into words
    tweet_refine = em.demojize(tweet_refine)
    return tweet_refine

In [7]:
# Get token length
def token_length(s):
    return len(s)

# Get the elmo embeddings
def elmo_tweet_embedder(tokens):
    
    tokens_input = tokens #load a tweet

    #create embedding
    embedding_tensor = elmo(inputs=tokens_input,
                            signature="default", as_dict=True)["elmo"] # <-- passing in a list instead of [word]

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        embedding = sess.run(embedding_tensor)
        return embedding

# Create "embedding lists" of equal size -- pad with empty characters, e.g. ""
# https://stackoverflow.com/questions/24066904/most-pythonic-way-to-extend-a-list-to-exactly-a-certain-length
def pad_list(some_list, target_len):
    return some_list[:target_len] + [""]*(target_len - len(some_list))


In [18]:
df['trans'] = df['Tweet'].apply(preprocess)
df['len'] = df['trans'].apply(token_length)
# df['embedding'] = df['trans'].apply(elmo_tweet_embedder)

# Find the tweet with the longest length
m = df['len'].max()

In [19]:
df.head()

,ID,Tweet,Affect Dimension,Intensity Score,trans,len
0,2017-En-10264,@xandraaa5 @amayaallyn6 shut up hashtags are c...,anger,0.562,shut up hashtags are cool offended,35
1,2017-En-10072,it makes me so fucking irate jesus. nobody is ...,anger,0.750,it makes me so fucking irate jesus. nobody is ...,109
2,2017-En-11383,Lol Adam the Bull with his fake outrage...,anger,0.417,Lol Adam the Bull with his fake outrage...,42
3,2017-En-11102,@THATSSHAWTYLO passed away early this morning ...,anger,0.354,passed away early this morning in a fast and ...,126
4,2017-En-11506,@Kristiann1125 lol wow i was gonna say really?...,anger,0.438,lol wow i was gonna say really?! haha have yo...,102


In [21]:
# takes about 3 min on 8 cpu cores for 1701 tweets
%%time
x = elmo_tweet_embedder(df['trans'].values)

CPU times: user 21min 26s, sys: 20.3 s, total: 21min 47s
Wall time: 2min 50s


In [42]:
np.shape(x)

(1701, 31, 1024)

In [51]:
X_train, X_test, y_train, y_test = train_test_split(x, df['Intensity Score'].values, test_size=0.33, random_state=42)

In [52]:
np.shape(X_train)

(1139, 31, 1024)

In [53]:
np.shape(y_train)

(1139,)

In [56]:
np.shape(y_train)

(1139,)

In [57]:
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional,Input, Embedding, Flatten
from keras import Model
from keras.optimizers import RMSprop

Using TensorFlow backend.
